In [ ]:
!git clone https://github.com/luca-medeiros/lang-segment-anything.git


Cloning into 'lang-segment-anything'...
remote: Enumerating objects: 376, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 376 (delta 115), reused 91 (delta 91), pack-reused 216 (from 1)
Receiving objects: 100% (376/376), 27.68 MiB | 60.95 MiB/s, done.
Resolving deltas: 100% (170/170), done.


In [ ]:
!git clone https://github.com/owenzlz/DeepImageBlending.git

Cloning into 'DeepImageBlending'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 249 (delta 26), reused 19 (delta 19), pack-reused 210 (from 1)
Receiving objects: 100% (249/249), 43.58 MiB | 71.86 MiB/s, done.
Resolving deltas: 100% (117/117), done.


In [ ]:
!pip install poetry

In [ ]:
%cd DeepImageBlending


/content/DeepImageBlending


In [ ]:
!poetry install

The "poetry.dev-dependencies" section is deprecated and will be removed in a future version. Use "poetry.group.dev.dependencies" instead.
Creating virtualenv deepimageblending-yMB-1Tj5-py3.10 in /root/.cache/pypoetry/virtualenvs
Installing dependencies from lock file

Package operations: 21 installs, 0 updates, 0 removals

  - Installing certifi (2022.12.7): Pending...
  - Installing charset-normalizer (2.1.1): Pending...
  - Installing idna (3.4): Pending...
  - Installing numpy (1.24.0): Pending...
  - Installing nvidia-cublas-cu11 (11.10.3.66): Pending...
  - Installing nvidia-cuda-nvrtc-cu11 (11.7.99): Pending...
  - Installing nvidia-cuda-runtime-cu11 (11.7.99): Pending...
  - Installing nvidia-cudnn-cu11 (8.5.0.96): Pending...
  - Installing pillow (9.3.0): Pending...
  - Installing typing-extensions (4.4.0): Pending...
  - Installing urllib3 (1.26.13): Pending...
  - Installing nvidia-cublas-cu11 (11.10.3.66): Pending...
  - Installing nvidia-cuda-nvrtc-cu11 (11.7.99): Pending..

In [ ]:

!sudo apt-get install python3.11-distutils
!sudo apt-get install python3.11-venv
!wget https://bootstrap.pypa.io/get-pip.py
!python3.11 get-pip.py


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'python3-distutils' instead of 'python3.11-distutils'
python3-distutils is already the newest version (3.10.8-1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.11-minimal libpython3.11-stdlib mailcap mime-support python3.11
  python3.11-minimal
Suggested packages:
  binfmt-support
The following NEW packages will be installed:
  libpython3.11-minimal libpython3.11-stdlib mailcap mime-support python3.11
  python3.11-minimal python3.11-venv
0 upgraded, 7 newly installed, 0 to remove and 49 not upgraded.
Need to get 8,303 kB of archives.
After this operation, 24.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 mailcap all 3.70+nmu1ubuntu1 [23.8 

In [ ]:
%cd ..

/


In [ ]:
!python3.11 -m venv langsamenv

In [ ]:
%cd lang-segment-anything

/content/lang-segment-anything


In [ ]:
!../langsamenv/bin/pip install -e .

Obtaining file:///content/lang-segment-anything
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/facebookresearch/segment-anything-2 (to revision c2ec8e14a185632b0a5d8b161928ceb50197eddc) to /tmp/pip-install-qaptwn34/sam-2_79c27fc2ea2c431cbdd3411cbf04a2e9
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything-2 /tmp/pip-install-qaptwn34/sam-2_79c27fc2ea2c431cbdd3411cbf04a2e9
  Running command git rev-parse -q --verify 'sha^c2ec8e14a185632b0a5d8b161928ceb50197eddc'
  Running command git fetch -q https://github.com/facebookresearch/segment-anything-2 c2ec8e14a185632b0a5d8b161928ceb50197eddc
  Running command git checkout -q c2ec8e14a185632b0a5d8b161928ceb50197eddc
  Resolved https://github.com/facebookresearch/segment-anything-2 to commit c2ec8e14a185

In [ ]:
!../langsamenv/bin/pip  install torch==2.4.1 torchvision==0.19.1 --extra-index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu124


In [ ]:
!../langsamenv/bin/pip install gradio

In [ ]:
script_content = """
# Packages
import numpy as np
import torch
import torch.optim as optim
from PIL import Image
from skimage.io import imsave
from torchvision.utils import save_image
from utils import compute_gt_gradient, make_canvas_mask, numpy2tensor, laplacian_filter_tensor, \
                  MeanShift, Vgg16, gram_matrix
import argparse
import pdb
import os
import imageio.v2 as iio
import torch.nn.functional as F



parser = argparse.ArgumentParser()
parser.add_argument('--source_file', type=str, default='data/1_source.png', help='path to the source image')
parser.add_argument('--mask_file', type=str, default='data/1_mask.png', help='path to the mask image')
parser.add_argument('--target_file', type=str, default='data/1_target.png', help='path to the target image')
parser.add_argument('--output_dir', type=str, default='results/1', help='path to output')
parser.add_argument('--ss', type=int, default=300, help='source image size')
parser.add_argument('--ts', type=int, default=512, help='target image size')
parser.add_argument('--x', type=int, default=200, help='vertical location (center)')
parser.add_argument('--y', type=int, default=235, help='vertical location (center)')
parser.add_argument('--gpu_id', type=int, default=0, help='GPU ID')
parser.add_argument('--num_steps', type=int, default=1000, help='Number of iterations in each pass')
parser.add_argument('--save_video', type=bool, default=False, help='save the intermediate reconstruction process')
opt = parser.parse_args()


os.makedirs(opt.output_dir, exist_ok = True)



###################################
########### First Pass ###########
###################################

# Inputs
source_file = opt.source_file
mask_file = opt.mask_file
target_file = opt.target_file

# Hyperparameter Inputs
gpu_id = opt.gpu_id
num_steps = opt.num_steps
ss = opt.ss; # source image size
ts = opt.ts # target image size
x_start = opt.x; y_start = opt.y # blending location

# Default weights for loss functions in the first pass
grad_weight = 1e4; style_weight = 1e4; content_weight = 1; tv_weight = 1e-6

# Load Images
source_img = np.array(Image.open(source_file).convert('RGB').resize((ss, ss)))
target_img = np.array(Image.open(target_file).convert('RGB').resize((ts, ts)))
mask_img = np.array(Image.open(mask_file).convert('L').resize((ss, ss)))
mask_img[mask_img>0] = 1

# Make Canvas Mask
canvas_mask = make_canvas_mask(x_start, y_start, target_img, mask_img)
canvas_mask = numpy2tensor(canvas_mask, gpu_id)
canvas_mask = canvas_mask.squeeze(0).repeat(3,1).view(3,ts,ts).unsqueeze(0)

# Compute Ground-Truth Gradients
gt_gradient = compute_gt_gradient(x_start, y_start, source_img, target_img, mask_img, gpu_id)

# Convert Numpy Images Into Tensors
source_img = torch.from_numpy(source_img).unsqueeze(0).transpose(1,3).transpose(2,3).float().to(gpu_id)
target_img = torch.from_numpy(target_img).unsqueeze(0).transpose(1,3).transpose(2,3).float().to(gpu_id)
input_img = torch.randn(target_img.shape).to(gpu_id)

mask_img = numpy2tensor(mask_img, gpu_id)
mask_img = mask_img.squeeze(0).repeat(3,1).view(3,ss,ss).unsqueeze(0)

# Define LBFGS optimizer
def get_input_optimizer(input_img):
    optimizer = optim.LBFGS([input_img.requires_grad_()])
    return optimizer
optimizer = get_input_optimizer(input_img)

# Define Loss Functions
mse = torch.nn.MSELoss()

# Import VGG network for computing style and content loss
mean_shift = MeanShift(gpu_id)
vgg = Vgg16().to(gpu_id)

# Save reconstruction process in a video
if opt.save_video:
    recon_process_video = iio.get_writer(os.path.join(opt.output_dir, 'recon_process.mp4'), format='FFMPEG', mode='I', fps=400)

run = [0]
while run[0] <= num_steps:

    def closure():
        # Composite Foreground and Background to Make Blended Image
        blend_img = torch.zeros(target_img.shape).to(gpu_id)
        blend_img = input_img*canvas_mask + target_img*(canvas_mask-1)*(-1)

        # Compute Laplacian Gradient of Blended Image
        pred_gradient = laplacian_filter_tensor(blend_img, gpu_id)

        # Compute Gradient Loss
        grad_loss = 0
        for c in range(len(pred_gradient)):
            grad_loss += mse(pred_gradient[c], gt_gradient[c])
        grad_loss /= len(pred_gradient)
        grad_loss *= grad_weight

        # Compute Style Loss
        target_features_style = vgg(mean_shift(target_img))
        target_gram_style = [gram_matrix(y) for y in target_features_style]

        blend_features_style = vgg(mean_shift(input_img))
        blend_gram_style = [gram_matrix(y) for y in blend_features_style]

        style_loss = 0
        for layer in range(len(blend_gram_style)):
            style_loss += mse(blend_gram_style[layer], target_gram_style[layer])
        style_loss /= len(blend_gram_style)
        style_loss *= style_weight


        # Compute Content Loss
        blend_obj = blend_img[:,:,int(x_start-source_img.shape[2]*0.5):int(x_start+source_img.shape[2]*0.5), int(y_start-source_img.shape[3]*0.5):int(y_start+source_img.shape[3]*0.5)]
        source_object_features = vgg(mean_shift(source_img*mask_img))
        blend_object_features = vgg(mean_shift(blend_obj*mask_img))
        content_loss = content_weight * mse(blend_object_features.relu2_2, source_object_features.relu2_2)
        content_loss *= content_weight

        # Compute TV Reg Loss
        tv_loss = torch.sum(torch.abs(blend_img[:, :, :, :-1] - blend_img[:, :, :, 1:])) + \
                   torch.sum(torch.abs(blend_img[:, :, :-1, :] - blend_img[:, :, 1:, :]))
        tv_loss *= tv_weight

        # Compute Total Loss and Update Image
        loss = grad_loss + style_loss + content_loss + tv_loss
        optimizer.zero_grad()
        loss.backward()

        # Write to output to a reconstruction video
        if opt.save_video:
            foreground = input_img*canvas_mask
            foreground = (foreground - foreground.min()) / (foreground.max() - foreground.min())
            background = target_img*(canvas_mask-1)*(-1)
            background = background / 255.0
            final_blend_img =  + foreground + background
            if run[0] < 200:
                # more frames for early optimization by repeatedly appending the frames
                for _ in range(10):
                    recon_process_video.append_data(final_blend_img[0].transpose(0,2).transpose(0,1).cpu().data.numpy())
            else:
                recon_process_video.append_data(final_blend_img[0].transpose(0,2).transpose(0,1).cpu().data.numpy())

        # Print Loss
        if run[0] % 1 == 0:
            print("run {}:".format(run))
            print('grad : {:4f}, style : {:4f}, content: {:4f}, tv: {:4f}'.format(\
                          grad_loss.item(), \
                          style_loss.item(), \
                          content_loss.item(), \
                          tv_loss.item()
                          ))
            print()

        run[0] += 1
        return loss

    optimizer.step(closure)

# clamp the pixels range into 0 ~ 255
input_img.data.clamp_(0, 255)

# Make the Final Blended Image
blend_img = torch.zeros(target_img.shape).to(gpu_id)
blend_img = input_img*canvas_mask + target_img*(canvas_mask-1)*(-1)
blend_img_np = blend_img.transpose(1,3).transpose(1,2).cpu().data.numpy()[0]

# Save image from the first pass
first_pass_img_file = os.path.join(opt.output_dir, 'first_pass.png')
imsave(first_pass_img_file, blend_img_np.astype(np.uint8))

###################################
########### Second Pass ###########
###################################

# Default weights for loss functions in the second pass
style_weight = 1e7
content_weight = 1
tv_weight = 1e-6
num_steps = opt.num_steps

# Dynamically get resolution from the first pass output
first_pass_img = np.array(Image.open(first_pass_img_file).convert('RGB'))
ss, ts = first_pass_img.shape[1], first_pass_img.shape[0]  # width, height

# Resize target image to match the resolution of the first pass
target_img = np.array(Image.open(target_file).convert('RGB').resize((ss, ts)))
first_pass_img = torch.from_numpy(first_pass_img).unsqueeze(0).transpose(1, 3).transpose(2, 3).float().to(gpu_id)
target_img = torch.from_numpy(target_img).unsqueeze(0).transpose(1, 3).transpose(2, 3).float().to(gpu_id)

first_pass_img = first_pass_img.contiguous()
target_img = target_img.contiguous()

# Define LBFGS optimizer
def get_input_optimizer(first_pass_img):
    optimizer = optim.LBFGS([first_pass_img.requires_grad_()])
    return optimizer

optimizer = get_input_optimizer(first_pass_img)

print('Optimizing...')
run = [0]
while run[0] <= num_steps:

    def closure():
        # Compute Style Loss
        target_features_style = vgg(mean_shift(target_img))
        target_gram_style = [gram_matrix(y) for y in target_features_style]
        blend_features_style = vgg(mean_shift(first_pass_img))
        blend_gram_style = [gram_matrix(y) for y in blend_features_style]
        style_loss = 0
        for layer in range(len(blend_gram_style)):
            style_loss += mse(blend_gram_style[layer], target_gram_style[layer])
        style_loss /= len(blend_gram_style)
        style_loss *= style_weight

        # Compute Content Loss
        content_features = vgg(mean_shift(first_pass_img))
        content_loss = content_weight * mse(blend_features_style.relu2_2, content_features.relu2_2)

        # Compute Total Loss and Update Image
        loss = style_loss + content_loss
        optimizer.zero_grad()
        loss.backward()

        # Print Loss
        if run[0] % 1 == 0:
            print("run {}:".format(run))
            print(' style : {:4f}, content: {:4f}'.format(
                          style_loss.item(),
                          content_loss.item()
                          ))
            print()

        run[0] += 1
        return loss

    optimizer.step(closure)

# Clamp pixel values to 0 ~ 255
first_pass_img.data.clamp_(0, 255)

# Save the final blended image from the second pass
input_img_np = first_pass_img.transpose(1, 3).transpose(1, 2).cpu().data.numpy()[0]
imsave(os.path.join(opt.output_dir, 'second_pass.png'), input_img_np.astype(np.uint8))

"""

# Save the script to `DeepImageBlending/run.py`
with open("/content/DeepImageBlending/run.py", "w") as file:
    file.write(script_content)

print("The script has been updated successfully!")


The script has been updated successfully!


In [ ]:
import os

# Define the scripts folder path
scripts_folder = "/content/scripts"
os.makedirs(scripts_folder, exist_ok=True)

# Define the file contents
files = {
    "app.py": """
import os
import sys
from PIL import Image

import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

from scripts.langsam_wrapper import LangSAMWrapper
from scripts.blend_wrapper import blend_images

# Paths to required directories
UPLOAD_DIR = "./uploads"
RESULT_DIR = "./results"
os.makedirs(UPLOAD_DIR, exist_ok=True)
os.makedirs(RESULT_DIR, exist_ok=True)

# Initialize LangSAM
lang_sam = LangSAMWrapper()

def process_images(source_path, target_path, text_prompt, size_factor, x, y):
    \"\"\"Full pipeline for object extraction and blending.\"\"\"
    mask_path = os.path.join(RESULT_DIR, "mask.png")
    output_dir = os.path.join(RESULT_DIR, "blend")

    # Step 1: Extract object using LangSAM
    print("Extracting object with LangSAM...")
    mask = lang_sam.get_mask(source_path, text_prompt)
    mask.save(mask_path)

    # Step 2: Blend the object onto the target image
    print("Blending object onto target image...")
    blended_img_path = blend_images(source_path, mask_path, target_path, output_dir, size_factor, x, y)

    print(f"Blending complete. Final image saved at: {blended_img_path}")
    return blended_img_path

if __name__ == "__main__":
    if len(sys.argv) != 7:
        print("Usage: python process_images.py <source_path> <target_path> <text_prompt> <size_factor> <x> <y>")
        sys.exit(1)

    source_path = sys.argv[1]
    target_path = sys.argv[2]
    text_prompt = sys.argv[3]
    size_factor = float(sys.argv[4])
    x = int(sys.argv[5])
    y = int(sys.argv[6])

    process_images(source_path, target_path, text_prompt, size_factor, x, y)
""",
    "blend_wrapper.py": """
import os
from subprocess import run
from PIL import Image

def blend_images(source_image, mask_image, target_image, output_dir, size_factor, x, y):
    os.makedirs(output_dir, exist_ok=True)

    with Image.open(source_image) as img:
        source_width, source_height = img.size

    with Image.open(target_image) as img:
        target_width, target_height = img.size

    ss = int(size_factor * max(source_width, source_height))
    ts = max(target_width, target_height)
    ss = min(ss, ts)

    deep_blending_script = "/content/DeepImageBlending/run.py"

    command = [
        "python",
        deep_blending_script,
        "--source_file", source_image,
        "--mask_file", mask_image,
        "--target_file", target_image,
        "--output_dir", output_dir,
        "--x", str(x),
        "--y", str(y),
        "--ss", str(ss),
        "--ts", str(ts),
    ]

    print("Executing command:", " ".join(command))
    run(command, check=True)

    return os.path.join(output_dir, "second_pass.png")
""",
    "combine_tools.py": """
import subprocess

langsam_python = "../langsamenv/bin/python"
deep_blending_python = "/usr/bin/python"

langsam_script = "/path/to/langsam_wrapper_script.py"
deep_blending_script = "/path/to/deep_image_blending_script.py"

print("Running LangSAM...")
subprocess.run([langsam_python, langsam_script], check=True)

print("Running Deep Image Blending...")
subprocess.run([deep_blending_python, deep_blending_script], check=True)
""",
    "langsam_wrapper.py": """
from PIL import Image
import numpy as np

class LangSAMWrapper:
    def __init__(self, model_type="sam2.1_hiera_small"):
        from lang_sam import LangSAM
        self.model = LangSAM(sam_type=model_type)

    def get_mask(self, image_path, text_prompt):
        image_pil = Image.open(image_path).convert("RGB")
        results = self.model.predict([image_pil], [text_prompt])
        print("LangSAM results:", results)

        if not results or "masks" not in results[0] or len(results[0]["masks"]) == 0:
            raise ValueError("No masks were generated. Please check your input image and prompt.")

        masks = results[0]["masks"]

        # Combine all masks into one
        combined_mask = np.zeros_like(masks[0], dtype=np.float32)
        for mask in masks:
            combined_mask = np.logical_or(combined_mask, mask).astype(np.float32)

        # Convert the combined mask to an image
        combined_mask_image = Image.fromarray((combined_mask * 255).astype(np.uint8))
        return combined_mask_image
"""
}

# Write the files to the scripts folder
for filename, content in files.items():
    with open(os.path.join(scripts_folder, filename), "w") as file:
        file.write(content.strip())

print("Scripts folder and files created successfully!")


Scripts folder and files created successfully!


In [ ]:
%cd ..

/content


In [ ]:
!mkdir uploads

In [ ]:
!langsamenv/bin/python scripts/app.py ./uploads/source.png ./uploads/target1.png "people." 0.6 450 300


Streaming output truncated to the last 5000 lines.
run [356]:
grad : 81961.914062, style : 26471.509766, content: 14444.843750, tv: 34.187870

run [357]:
grad : 81507.265625, style : 26021.890625, content: 14319.310547, tv: 34.184170

run [358]:
grad : 80921.421875, style : 26213.218750, content: 14163.239258, tv: 34.180649

run [359]:
grad : 80492.312500, style : 25614.275391, content: 14153.785156, tv: 34.178738

run [360]:
grad : 80379.117188, style : 25670.542969, content: 14032.556641, tv: 34.176254

run [361]:
grad : 80192.812500, style : 25386.468750, content: 14066.638672, tv: 34.176678

run [362]:
grad : 80010.773438, style : 25279.476562, content: 14041.490234, tv: 34.175785

run [363]:
grad : 79449.320312, style : 24966.115234, content: 13882.069336, tv: 34.171474

run [364]:
grad : 79063.585938, style : 24583.613281, content: 13790.549805, tv: 34.167889

run [365]:
grad : 78682.742188, style : 24877.736328, content: 13613.434570, tv: 34.164368

run [366]:
grad : 78252.14062